<a href="https://colab.research.google.com/github/salma71/WiDS_kaggle_competition/blob/master/WiDS_project_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#unzip the folder
!mkdir train

Saving kaggle.json to kaggle.json


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c widsdatathon2021

  0% 0.00/1.60M [00:00<?, ?B/s]
100% 1.60M/1.60M [00:00<00:00, 51.7MB/s]
  0% 0.00/28.2k [00:00<?, ?B/s]
100% 28.2k/28.2k [00:00<00:00, 47.4MB/s]
  0% 0.00/90.0k [00:00<?, ?B/s]
100% 90.0k/90.0k [00:00<00:00, 75.7MB/s]
 42% 9.00M/21.5M [00:00<00:00, 13.8MB/s]
100% 21.5M/21.5M [00:00<00:00, 27.6MB/s]
  0% 0.00/59.0 [00:00<?, ?B/s]
100% 59.0/59.0 [00:00<00:00, 63.2kB/s]


In [ ]:
!unzip TrainingWiDS2021.csv.zip -d train

Archive:  TrainingWiDS2021.csv.zip
  inflating: train/TrainingWiDS2021.csv  


Now the the train directory has the training dataset. I will add the test data into another folder called test. 

In [ ]:
#unzip the folder
!mkdir test
!unzip UnlabeledWiDS2021.csv.zip -d test

Archive:  UnlabeledWiDS2021.csv.zip
  inflating: test/UnlabeledWiDS2021.csv  


Import important and essential libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt # visualization
!pip install seaborn as sns -q --upgrade pip # visualization with seaborn v0.11.1
import seaborn as sns # visualization
import missingno as msno # missing values pattern visualization
#set seed
import random
seed = 41
!pip install dabl -q --upgrade pip
import dabl # quick exploration and model assessment

import warnings # supress warnings
warnings.filterwarnings('ignore')

# set pandas display option
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

     |████████████████████████████████| 1.5MB 12.0MB/s 
     |████████████████████████████████| 558 kB 12.5 MB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 


In [ ]:
!pwd

/content


In [ ]:
# Load the data 
train_df = pd.read_csv('/content/train/TrainingWiDS2021.csv')
test_df = pd.read_csv('/content/test/UnlabeledWiDS2021.csv')
data_dictionary_df = pd.read_csv("/content/DataDictionaryWiDS2021.csv")

# Drop first column because it is identical to index
# train_df.drop(columns = ['Unnamed: 0', 'encounter_id', 'hospital_id'], axis = 1, inplace = True)
# test_df.drop(['Unnamed: 0', 'encounter_id', 'hospital_id'], axis = 1, inplace = True)

# display the dataset
train_df.head().style.set_caption('Sample of training data')

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,1,214826,118,68.000000,22.732803,0,Caucasian,M,180.300000,Floor,Floor,92,admit,CTICU,0.541667,0,73.900000,2.300000,113.000000,502.010000,0,0,0.400000,31.000000,2.510000,nan,3.000000,6.000000,0.000000,4.000000,168.000000,118.000000,27.400000,0,40.000000,nan,nan,nan,nan,36.000000,134.000000,39.300000,nan,0,14.100000,46.000000,32.000000,68.000000,37.000000,68.000000,37.000000,119.000000,72.000000,66.000000,40.000000,89.000000,46.000000,89.000000,46.000000,34.000000,10.000000,100.000000,74.000000,122.000000,64.000000,131.000000,73.000000,131.000000,73.000000,39.900000,37.200000,nan,nan,68.000000,63.000000,68.000000,63.000000,119.000000,108.000000,nan,nan,86.000000,85.000000,86.000000,85.000000,26.000000,18.000000,100.000000,74.000000,nan,nan,131.000000,115.000000,131.000000,115.000000,39.500000,37.500000,2.300000,2.300000,0.400000,0.400000,31.000000,30.000000,8.500000,7.400000,2.510000,2.230000,168.000000,109.000000,19.000000,15.000000,8.900000,8.900000,27.400000,27.400000,nan,nan,1.300000,1.000000,233.000000,233.000000,4.000000,3.400000,136.000000,134.000000,14.100000,14.100000,n

In [ ]:
# display the dataset
test_df.head().style.set_caption('Sample of testing data')

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,1,144740,10141,72,nan,0,Caucasian,F,152.400000,Floor,Accident & Emergency,82,admit,Med-Surg ICU,0.015278,0,nan,2.800000,110.000000,104.010000,0,0,1.900000,44.000000,1.490000,nan,4.000000,6.000000,0.000000,5.000000,97.000000,38.000000,39.900000,0,54.000000,nan,nan,nan,nan,31.000000,130.000000,36.400000,nan,0,5.400000,nan,nan,104.000000,40.000000,104.000000,40.000000,66.000000,38.000000,nan,nan,123.000000,54.000000,123.000000,54.000000,35.000000,15.000000,100.000000,96.000000,nan,nan,149.000000,76.000000,149.000000,76.000000,38.700000,35.600000,nan,nan,59.000000,59.000000,59.000000,59.000000,46.000000,44.000000,nan,nan,80.000000,80.000000,80.000000,80.000000,19.000000,19.000000,97.000000,96.000000,nan,nan,116.000000,116.000000,116.000000,116.000000,36.400000,36.400000,2.800000,2.800000,1.900000,1.900000,44.000000,40.000000,9.800000,9.300000,1.490000,1.440000,104.000000,97.000000,26.000000,23.000000,14.500000,14.200000,42.800000,39.900000,nan,nan,nan,nan,173.000000,173.000000,5.700000,4.900000,132.000000,130.000000,5.600000,5.400000,2.800000,2.800000,1.900000,1.900000,40.000000,40.000000,9.800000,

## Data Engineering 

In [ ]:
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import pickle
from scipy import stats, special
import pandas as pd
import numpy as np
import os
import warnings
warnings.simplefilter("ignore")

In [ ]:
#remove columns with large number of nan values
def remove_nan_cols (df, threshold=0.5):
    nan_cols = []
    for col in df.columns:
        nan_ratio = df[col].isnull().sum() / df.shape[0]
        if nan_ratio >= threshold:
            nan_cols.append(col)
    df = df.drop(nan_cols, axis=1)
    return df

In [ ]:
#filling missing values based on linear regression and the most correlated variables
def fillna_using_linear_model(df, feature_cols):

    correl = df[feature_cols].corr()

    for col in tqdm(feature_cols):
        nan_ratio = df[col].isnull().sum() / df.shape[0]
        if nan_ratio > 0:
            best_nan_ratio = nan_ratio
            best_col = None
            for id in correl.loc[(correl[col] > 0.7) | (correl[col] < -0.7), col].index:
                nan_temp_ratio = df[id].isnull().sum() / df.shape[0]
                if best_nan_ratio > nan_temp_ratio:
                    best_nan_ratio = nan_temp_ratio
                    best_col = id
            if best_col != None:
                sub = df[[col, best_col]].copy()
                sub = sub.dropna()
                reg = LinearRegression(fit_intercept=True).fit(np.expand_dims(sub[best_col], axis=1), sub[col])
                print(reg.score(np.expand_dims(sub[best_col], axis=1), sub[col]))
                if reg.score(np.expand_dims(sub[best_col], axis=1), sub[col])>0.7:
                    if df.loc[(~df[best_col].isnull()) & (df[col].isnull()), col].shape[0] > 0:
                        df.loc[(~df[best_col].isnull()) & (df[col].isnull()), col] = \
                            reg.predict(np.expand_dims(
                                df.loc[(~df[best_col].isnull()) & (df[col].isnull()), best_col], axis=1))

    return df

In [ ]:
def feature_generation(df):

    agg = df['icu_id'].value_counts().to_dict()
    df['icu_id_counts'] = np.log1p(df['icu_id'].map(agg))
    agg = df['age'].value_counts().to_dict()
    df['age_counts'] = np.log1p(df['age'].map(agg))

    df['nan_counts'] = df.isnull().sum(axis=1)

    df['sq_age'] = df['age'].values ** 2
    df['sq_bmi'] = df['bmi'].values ** 2
    df['bmi_age'] = df['bmi'].values / df['age'].values
    df['weight_age'] = df['age'].values / df['weight'].values

    df['comorbidity_score'] = df['aids'].values * 23 + df['cirrhosis'].values * 4 + df['diabetes_mellitus'].values * 1 + \
                                 df['hepatic_failure'].values * 16 + df['immunosuppression'].values * 10 + \
                              df['leukemia'].values * 10 + df['lymphoma'].values * 13 + df['solid_tumor_with_metastasis'].values * 11
    
    
    #source https://www.omnicalculator.com/health/risk-dm
    df['diabete_risk'] = 100 / (1 + np.exp(-1*(0.028*df['age'].values + 0.661*np.where(df['gender'].values=="M", 1, 0) +
                                               0.412 * np.where(df['ethnicity'].values=="Native American", 0, 1) +
                                               0.079 * df['glucose_apache'].values + 0.018 * df['d1_diasbp_max'].values +
                                               0.07 * df['bmi'].values + 0.481 * df['cirrhosis'].values - 13.415)))

    df['gcs_sum'] = df['gcs_eyes_apache'].values + df['gcs_motor_apache'].values + df['gcs_verbal_apache'].values
    df['apache_2_diagnosis_type'] = df.apache_2_diagnosis.round(-1).fillna(-100).astype('int32')
    df['apache_3j_diagnosis_type'] = df.apache_3j_diagnosis.round(-2).fillna(-100).astype('int32')
    df['bmi_type'] = df.bmi.fillna(0).apply(lambda x: 5 * (round(int(x) / 5)))
    df['height_type'] = df.height.fillna(0).apply(lambda x: 5 * (round(int(x) / 5)))
    df['weight_type'] = df.weight.fillna(0).apply(lambda x: 5 * (round(int(x) / 5)))
    df['age_type'] = df.age.fillna(0).apply(lambda x: 10 * (round(int(x) / 10)))
    df['gcs_sum_type'] = df.gcs_sum.fillna(0).apply(lambda x: 2.5 * (round(int(x) / 2.5))).divide(2.5)
    df['apache_3j_diagnosis_x'] = df['apache_3j_diagnosis'].astype('str').str.split('.', n=1, expand=True)[0]
    df['apache_2_diagnosis_x'] = df['apache_2_diagnosis'].astype('str').str.split('.', n=1, expand=True)[0]
    df['apache_3j_diagnosis_split1'] = np.where(df['apache_3j_diagnosis'].isna(), np.nan,
                                                   df['apache_3j_diagnosis'].astype('str').str.split('.', n=1,
                                                                                                        expand=True)[1])
    df['apache_2_diagnosis_split1'] = np.where(df['apache_2_diagnosis'].isna(), np.nan,
                                                  df['apache_2_diagnosis'].apply(lambda x: x % 10))

    IDENTIFYING_COLS = ['age_type', 'gcs_sum_type', 'ethnicity', 'gender', 'bmi_type']
    df['profile'] = df[IDENTIFYING_COLS].apply(lambda x: hash(tuple(x)), axis=1)

    df["diff_bmi"] = df['bmi'].copy()
    df['bmi'] = df['weight'].values / ((df['height'].values / 100) ** 2)
    df["diff_bmi"] = df["diff_bmi"].values - df['bmi'].values

    df['pre_icu_los_days'] = df['pre_icu_los_days'].apply(lambda x: special.expit(x))

    d_cols = [c for c in df.columns if (c.startswith("d1"))]
    h_cols = [c for c in df.columns if (c.startswith("h1"))]
    df["dailyLabs_row_nan_count"] = df[d_cols].isna().sum(axis=1)
    df["hourlyLabs_row_nan_count"] = df[h_cols].isna().sum(axis=1)
    df["diff_labTestsRun_daily_hourly"] = df["dailyLabs_row_nan_count"].values - df["hourlyLabs_row_nan_count"].values

    lab_col = [c for c in df.columns if ((c.startswith("h1")) | (c.startswith("d1")))]
    lab_col_names = list(set(list(map(lambda i: i[3: -4], lab_col))))

    first_h = []
    for v in tqdm(lab_col_names):
        colsx = [x for x in df.columns if v in x]
        df[v + "_nans"] = df.loc[:, colsx].isna().sum(axis=1)
        df[v + "_d1_h1_max_eq"] = (df[f"d1_{v}_max"] == df[f"h1_{v}_max"]).astype(np.int8)
        df[v + "_d1_h1_min_eq"] = (df[f"d1_{v}_min"] == df[f"h1_{v}_min"]).astype(np.int8)

        for freq in ['h1', 'd1']:
            df[v + f"_{freq}_value_range"] = df[f"{freq}_{v}_max"].subtract(df[f"{freq}_{v}_min"])
            df[v + f"_{freq}_zero_range"] = (df[v + f"_{freq}_value_range"] == 0).astype(np.int8)
            df[v + f"_{freq}_mean"] =np.nanmean(df[[f"{freq}_{v}_max", f"{freq}_{v}_min"]].values, axis=1)
            df[v + f"_{freq}_std"] = np.nanstd(df[[f"{freq}_{v}_max", f"{freq}_{v}_min"]].values, axis=1)

            for g in ['apache_3j_diagnosis', 'profile', 'icu_id']:
                for m in ['max', 'min']:
                    temp = df[[g, f"{freq}_{v}_{m}"]].groupby(g)
                    df[v + f"_{freq}_{m}_{g}_mean"] = temp.transform('mean')
                    df[v + f"_{freq}_{m}_{g}_diff"] = df[v + f"_{freq}_{m}_{g}_mean"].subtract(df[f"{freq}_{v}_{m}"])
                    df[v + f"_{freq}_{m}_{g}_std"] = temp.transform('std')
                    df[v + f"_{freq}_{m}_{g}_norm_std"] = df[v + f"_{freq}_{m}_{g}_std"].div(df[f"{freq}_{v}_{m}"])
                    df[v + f"_{freq}_{m}_{g}_rank"] =temp.transform('rank')
                    df[v + f"_{freq}_{m}_{g}_count"] = temp.transform('count')
                    df[v + f"_{freq}_{m}_{g}_norm_rank"] = df[v + f"_{freq}_{m}_{g}_rank"].div(df[v + f"_{freq}_{m}_{g}_count"])
                    df[v + f"_{freq}_{m}_{g}_skew"] = temp.transform('skew')

            if v + "_apache" in colsx:
                for m in ['max', 'min']:
                    df[v + f"_apache_{freq}_{m}_ratio"] = df[f"{freq}_{v}_{m}"].div(df[v + "_apache"])

            for m in ['max', 'min']:
                df[f"{freq}_{v}_{m}_bmi_ratio"] = df[f"{freq}_{v}_{m}"].div(df['bmi'])
                df[f"{freq}_{v}_{m}_weight_ratio"] = df[f"{freq}_{v}_{m}"].div(df['weight'])

        df[v + "_range_between_d_h"] = df[v + "_d1_mean"].values - df[v + "_h1_mean"].values
        df[v + "_d1_h1_mean"] = np.nanmean(df[[f"d1_{v}_max", f"d1_{v}_min", f"h1_{v}_max", f"h1_{v}_min"]].values, axis=1)
        df[v + "_d1_h1_std"] = np.nanstd(df[[f"d1_{v}_max", f"d1_{v}_min", f"h1_{v}_max", f"h1_{v}_min"]].values, axis=1)

        df[v + "_tot_change_value_range_normed"] = abs((df[v + "_d1_value_range"].div(df[v + "_h1_value_range"])))
        df[v + "_started_after_firstHour"] = ((df[f"h1_{v}_max"].isna()) & (df[f"h1_{v}_min"].isna())) & (~df[f"d1_{v}_max"].isna())
        first_h.append(v + "_started_after_firstHour")
        df[v + "_day_more_extreme"] = ((df[f"d1_{v}_max"] > df[f"h1_{v}_max"]) | (df[f"d1_{v}_min"] < df[f"h1_{v}_min"]))
        df[v + "_day_more_extreme"].fillna(False)

    df["total_Tests_started_After_firstHour"] = df[first_h].sum(axis=1)

    df['diasbp_indicator'] = (
            (df['d1_diasbp_invasive_max'] == df['d1_diasbp_max']) & (
            df['d1_diasbp_noninvasive_max'] == df['d1_diasbp_invasive_max']) |
            (df['d1_diasbp_invasive_min'] == df['d1_diasbp_min']) & (
                    df['d1_diasbp_noninvasive_min'] == df['d1_diasbp_invasive_min']) |
            (df['h1_diasbp_invasive_max'] == df['h1_diasbp_max']) & (
                    df['h1_diasbp_noninvasive_max'] == df['h1_diasbp_invasive_max']) |
            (df['h1_diasbp_invasive_min'] == df['h1_diasbp_min']) & (
                    df['h1_diasbp_noninvasive_min'] == df['h1_diasbp_invasive_min'])).astype(np.int8)

    df['mbp_indicator'] = (
            (df['d1_mbp_invasive_max'] == df['d1_mbp_max']) & (
            df['d1_mbp_noninvasive_max'] == df['d1_mbp_invasive_max']) |
            (df['d1_mbp_invasive_min'] == df['d1_mbp_min']) & (
                    df['d1_mbp_noninvasive_min'] == df['d1_mbp_invasive_min']) |
            (df['h1_mbp_invasive_max'] == df['h1_mbp_max']) & (
                    df['h1_mbp_noninvasive_max'] == df['h1_mbp_invasive_max']) |
            (df['h1_mbp_invasive_min'] == df['h1_mbp_min']) & (
                    df['h1_mbp_noninvasive_min'] == df['h1_mbp_invasive_min'])
    ).astype(np.int8)

    df['sysbp_indicator'] = (
            (df['d1_sysbp_invasive_max'] == df['d1_sysbp_max']) & (
            df['d1_sysbp_noninvasive_max'] == df['d1_sysbp_invasive_max']) |
            (df['d1_sysbp_invasive_min'] == df['d1_sysbp_min']) & (
                    df['d1_sysbp_noninvasive_min'] == df['d1_sysbp_invasive_min']) |
            (df['h1_sysbp_invasive_max'] == df['h1_sysbp_max']) & (
                    df['h1_sysbp_noninvasive_max'] == df['h1_sysbp_invasive_max']) |
            (df['h1_sysbp_invasive_min'] == df['h1_sysbp_min']) & (
                    df['h1_sysbp_noninvasive_min'] == df['h1_sysbp_invasive_min'])
    ).astype(np.int8)

    df['d1_mbp_invnoninv_max_diff'] = df['d1_mbp_invasive_max'].div(df['d1_mbp_noninvasive_max'])
    df['h1_mbp_invnoninv_max_diff'] = df['h1_mbp_invasive_max'].div(df['h1_mbp_noninvasive_max'])
    df['d1_mbp_invnoninv_min_diff'] = df['d1_mbp_invasive_min'].div(df['d1_mbp_noninvasive_min'])
    df['h1_mbp_invnoninv_min_diff'] = df['h1_mbp_invasive_min'].div(df['h1_mbp_noninvasive_min'])
    df['d1_diasbp_invnoninv_max_diff'] = df['d1_diasbp_invasive_max'].div(df['d1_diasbp_noninvasive_max'])
    df['h1_diasbp_invnoninv_max_diff'] = df['h1_diasbp_invasive_max'].div(df['h1_diasbp_noninvasive_max'])
    df['d1_diasbp_invnoninv_min_diff'] = df['d1_diasbp_invasive_min'].div(df['d1_diasbp_noninvasive_min'])
    df['h1_diasbp_invnoninv_min_diff'] = df['h1_diasbp_invasive_min'].div(df['h1_diasbp_noninvasive_min'])
    df['d1_sysbp_invnoninv_max_diff'] = df['d1_sysbp_invasive_max'].div(df['d1_sysbp_noninvasive_max'])
    df['h1_sysbp_invnoninv_max_diff'] = df['h1_sysbp_invasive_max'].div(df['h1_sysbp_noninvasive_max'])
    df['d1_sysbp_invnoninv_min_diff'] = df['d1_sysbp_invasive_min'].div(df['d1_sysbp_noninvasive_min'])
    df['h1_sysbp_invnoninv_min_diff'] = df['h1_sysbp_invasive_min'].div(df['h1_sysbp_noninvasive_min'])

    more_extreme_cols = [c for c in df.columns if (c.endswith("_day_more_extreme"))]
    df["total_day_more_extreme"] = df[more_extreme_cols].sum(axis=1)
    df["d1_resprate_div_mbp_min"] = df["d1_resprate_min"].div(df["d1_mbp_min"])
    df["d1_resprate_div_sysbp_min"] = df["d1_resprate_min"].div(df["d1_sysbp_min"])
    df["d1_lactate_min_div_diasbp_min"] = df["d1_lactate_min"].div(df["d1_diasbp_min"])
    df["d1_heartrate_min_div_d1_sysbp_min"] = df["d1_heartrate_min"].div(df["d1_sysbp_min"])
    df["total_chronic"] = df[["aids", "cirrhosis", 'hepatic_failure']].sum(axis=1)
    df["total_cancer_immuno"] = df[['immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']].sum(axis=1)
    df["has_complicator"] = df[["aids", "cirrhosis", 'hepatic_failure','immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']].max(axis=1)

    df['apache_3j'] = np.where(df['apache_3j_diagnosis_type'] < 0, np.nan,
                               np.where(df['apache_3j_diagnosis_type'] < 200, 'Cardiovascular',
                                        np.where(df['apache_3j_diagnosis_type'] < 400, 'Respiratory',
                                                 np.where(df['apache_3j_diagnosis_type'] < 500, 'Neurological',
                                                          np.where(df['apache_3j_diagnosis_type'] < 600, 'Sepsis',
                                                                   np.where(df['apache_3j_diagnosis_type'] < 800,
                                                                            'Trauma',
                                                                            np.where(
                                                                                df['apache_3j_diagnosis_type'] < 900,
                                                                                'Haematological',
                                                                                np.where(df[
                                                                                             'apache_3j_diagnosis_type'] < 1000,
                                                                                         'Renal/Genitourinary',
                                                                                         np.where(df[
                                                                                                      'apache_3j_diagnosis_type'] < 1200,
                                                                                                  'Musculoskeletal/Skin disease',
                                                                                                  'Operative Sub-Diagnosis Codes')))))))))


    return df

In [ ]:
#remove columns with low variation
def remove_feature_with_low_var(df, threshold=0.1):
    for col in df.columns:
        if df[col].std() < threshold:
            df = df.drop([col], axis=1)
    return df

In [ ]:

train_df['which_data'] = 'train'
test_df['which_data'] = 'test'
test_df['diabetes_mellitus'] = 0

df = pd.concat([train_df, test_df[train_df.columns]], axis=0)

del train_df, test_df

df['age'] = np.where(df['age'].values==0, np.nan, df['age'].values)
df = df.replace([np.inf, -np.inf], np.nan)

min_max_feats=[f[:-4] for f in df.columns if f[-4:]=='_min']
for col in min_max_feats:
    df.loc[df[f'{col}_min'] > df[f'{col}_max'], [f'{col}_min', f'{col}_max']] = \
        df.loc[df[f'{col}_min'] > df[f'{col}_max'], [f'{col}_max', f'{col}_min']].values

nan_col = False
if nan_col ==True:
    df = remove_nan_cols(df, threshold=0.75)

cont_cols = []
for col in df.columns:
    if df[col].dtype=='float64':
        cont_cols.append(col)

fillna=True
if fillna==True:
    df = fillna_using_linear_model(df, cont_cols)

df = feature_generation(df)
df = df.replace([np.inf, -np.inf], np.nan)
    
cats = ['elective_surgery', 'icu_id', 'arf_apache', 'intubated_apache', 'ventilated_apache', 'cirrhosis',
    'hepatic_failure', 'immunosuppression', 'leukemia', 'solid_tumor_with_metastasis', 'apache_3j_diagnosis_x',
    'apache_2_diagnosis_x', 'apache_3j', 'apache_3j_diagnosis_split1', 'apache_2_diagnosis_split1', 'gcs_sum_type',
    'hospital_admit_source', 'glucose_rate', 'glucose_wb', 'gcs_eyes_apache', 'glucose_normal', 'total_cancer_immuno',
    'gender', 'total_chronic', 'icu_stay_type', 'apache_2_diagnosis_type', 'apache_3j_diagnosis_type']


df['hospital_admit_source'] = df['hospital_admit_source'].replace({'Other ICU': 'ICU','ICU to SDU':'SDU',
                                                                          'Step-Down Unit (SDU)': 'SDU',
                                                                          'Other Hospital':'Other','Observation': 'Recovery Room',
                                                                          'Acute Care/Floor': 'Acute Care'})

drop_cols = []
features = df.columns
features = [f for f in features if f not in cats]
for col in features:
    if df[col].dtype!='object' and col!='diabetes_mellitus':
        if df[col].std()==0 or df[col].std()==np.nan:
            drop_cols.append(col)

df = df.drop(drop_cols, axis=1)

cats = [f for f in cats if f in df.columns]
train_df = df.loc[df['which_data']=='train']
test_df = df.loc[df['which_data']=='test']
train_df = train_df.drop(['which_data'], axis=1)
test_df = test_df.drop(['which_data'], axis=1)

for col in train_df.select_dtypes(exclude = np.number).columns.tolist():
    train_only = list(set(train_df[col].unique()) - set(test_df[col].unique()))
    test_only = list(set(test_df[col].unique()) - set(train_df[col].unique()))
    both = list(set(test_df[col].unique()).union(set(train_df[col].unique())))
    train_df.loc[train_df[col].isin(train_only), col] = np.nan
    test_df.loc[test_df[col].isin(test_only), col] = np.nan

    test_df[col] = test_df[col].astype('str')
    train_df[col] = train_df[col].astype('str')
    le = LabelEncoder()
    le.fit(pd.concat([train_df[col], test_df[col]]))

    train_df[col] = le.transform(train_df[col])
    test_df[col] = le.transform(test_df[col])


for col in tqdm(cats):
    train_only = list(set(train_df[col].unique()) - set(test_df[col].unique()))
    test_only = list(set(test_df[col].unique()) - set(train_df[col].unique()))
    both = list(set(test_df[col].unique()).union(set(train_df[col].unique())))
    train_df.loc[train_df[col].isin(train_only), col] = np.nan
    test_df.loc[test_df[col].isin(test_only), col] = np.nan
    try:
        le = LabelEncoder()
        le.fit(pd.concat([train_df[col], test_df[col]]))
        train_df[col] = le.transform(train_df[col])
        test_df[col] = le.transform(test_df[col])
    except:
        test_df[col] = test_df[col].astype('str').fillna('-1')
        train_df[col] = train_df[col].astype('str').fillna('-1')
        le = LabelEncoder()
        le.fit(pd.concat([train_df[col], test_df[col]]))
        train_df[col] = le.transform(train_df[col])
        test_df[col] = le.transform(test_df[col])

    temp = pd.concat([train_df[[col]], test_df[[col]]], axis=0)
    temp_mapping = temp.groupby(col).size()/len(temp)
    temp['enc'] = temp[col].map(temp_mapping)
    temp['enc'] = stats.rankdata(temp['enc'])
    temp = temp.reset_index(drop=True)
    train_df[f'rank_frqenc_{col}'] = temp[['enc']].values[:train_df.shape[0]]
    test_df[f'rank_frqenc_{col}'] = temp[['enc']].values[train_df.shape[0]:]
    test_df[col] = test_df[col].astype('category')
    train_df[col] = train_df[col].astype('category')


feature_cols = list(train_df)
feature_cols.remove('diabetes_mellitus')
feature_cols.remove('encounter_id')
feature_cols.remove('hospital_id')

cont_cols = feature_cols
cont_cols = [f for f in cont_cols if f not in cats]

for col in cont_cols:
    r = stats.ks_2samp(train_df[col].dropna(), test_df[col].dropna())
    print(col, r)
    if r[0] >= 0.05:
        cont_cols.remove(col)

print("Number of input variables:", len(cont_cols)+len(cats))

train_df = train_df[cont_cols+cats+['encounter_id', 'diabetes_mellitus']]
test_df = test_df[cont_cols+cats+['encounter_id']]



  8%|▊         | 13/157 [00:00<00:02, 61.71it/s]

0.7709981024670356
0.8907221897585749
0.9936445964621089
0.9820163459813478
0.9872263997304246
0.8516423523695633


 17%|█▋        | 27/157 [00:00<00:02, 64.11it/s]

0.6560436678586896
0.8261585034190566
0.6617421582355307
0.6617421582355307
0.534975529700368
0.7753871354181349
0.6162324332466131
0.9376387077854774
0.5633952514492098


 29%|██▊       | 45/157 [00:00<00:01, 74.98it/s]

...

0.9546757775766881
0.622968098326483
0.548690176399227
0.7348462577021715
0.7326684258586742
0.7995458774119808
0.7972937149039434
0.8457203642295851


100%|██████████| 157/157 [00:02<00:00, 67.44it/s]


0.7718520535914031
0.716131604975343
0.7702810374307323
0.775666191208561
0.5926355675399744
0.6562037111038768
0.5892430182647266


100%|██████████| 24/24 [00:42<00:00,  1.76s/it]


Unnamed: 0 Ks_2sampResult(statistic=0.9213718816506219, pvalue=0.0)
bmi Ks_2sampResult(statistic=0.008181292232846027, pvalue=0.6133596347230488)
ethnicity Ks_2sampResult(statistic=0.01608563205883634, pvalue=0.014745230884218587)
height Ks_2sampResult(statistic=0.01696180304353967, pvalue=0.00994184361806165)
icu_admit_source Ks_2sampResult(statistic=0.00972612608321377, pvalue=0.3307061376161604)
icu_type Ks_2sampResult(statistic=0.0055594424561769, pvalue=0.9310728414661898)
pre_icu_los_days Ks_2sampResult(statistic=0.044066982535301, pvalue=1.9840081696877508e-16)
weight Ks_2sampResult(statistic=0.010001958074934092, pvalue=0.34983188372974594)
albumin_apache Ks_2sampResult(statistic=0.014052498110478678, pvalue=0.34557991872660704)
apache_2_diagnosis Ks_2sampResult(statistic=0.019273467789830245, pvalue=0.001948742353513992)
apache_3j_diagnosis Ks_2sampResult(statistic=0.01478847724011123, pvalue=0.032900634972469756)
apache_post_operative Ks_2sampResult(statistic=0.01254854540314

In [ ]:
# create a directory to store the cleaned data!
!mkdir clean_data

In [ ]:
with open('clean_data/train_df.csv', 'wb') as fout:
    pickle.dump(train_df, fout)

with open('clean_data/test_df.pickle', 'wb') as fout:
    pickle.dump(test_df, fout)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# move the data including pkl files into drive to use it later in different notebook
import os
os.listdir('.')

['.config',
 'train',
 'TrainingWiDS2021.csv.zip',
 'SampleSubmissionWiDS2021.csv',
 'SolutionTemplateWiDS2021.csv',
 'drive',
 'clean_data',
 'test',
 'DataDictionaryWiDS2021.csv',
 'data',
 'UnlabeledWiDS2021.csv.zip',
 'kaggle.json',
 'sample_data']

In [ ]:
!ls clean_data

test_df.pickle	train_df.pickle


In [ ]:
!ls /content/drive/MyDrive/

In [ ]:
mv /content/clean_data/ /content/drive/MyDrive/Colab\ Notebooks